In [1]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Screen settings
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Drag the Green Circles into the Black Square")

# Colors
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)

# Fonts
font = pygame.font.SysFont(None, 36)

# Game objects
NUM_CIRCLES = 10
CIRCLE_RADIUS = 15
SQUARE_SIZE = 80
SQUARE_POS = (WIDTH - SQUARE_SIZE - 50, HEIGHT//2 - SQUARE_SIZE//2)

# Generate random circle positions
circles = []
for _ in range(NUM_CIRCLES):
    x = random.randint(CIRCLE_RADIUS, WIDTH - CIRCLE_RADIUS - SQUARE_SIZE - 100)
    y = random.randint(CIRCLE_RADIUS, HEIGHT - CIRCLE_RADIUS)
    circles.append({'pos': [x, y], 'dragging': False})

# Score
score = 0

# Main loop
running = True
clock = pygame.time.Clock()

while running:
    screen.fill(WHITE)

    # Draw black square
    pygame.draw.rect(screen, BLACK, (*SQUARE_POS, SQUARE_SIZE, SQUARE_SIZE))

    # Draw score
    score_text = font.render(f"Score: {score}", True, BLACK)
    screen.blit(score_text, (WIDTH - 180, 20))

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            for circle in circles:
                cx, cy = circle['pos']
                if (pygame.math.Vector2(pygame.mouse.get_pos()) - pygame.math.Vector2(cx, cy)).length() <= CIRCLE_RADIUS:
                    circle['dragging'] = True
        elif event.type == pygame.MOUSEBUTTONUP:
            for circle in circles:
                if circle['dragging']:
                    circle['dragging'] = False
                    cx, cy = circle['pos']
                    sx, sy = SQUARE_POS
                    if sx <= cx <= sx + SQUARE_SIZE and sy <= cy <= sy + SQUARE_SIZE:
                        circles.remove(circle)
                        score += 1
        elif event.type == pygame.MOUSEMOTION:
            for circle in circles:
                if circle['dragging']:
                    circle['pos'] = list(event.pos)

    # Draw circles
    for circle in circles:
        pygame.draw.circle(screen, GREEN, circle['pos'], CIRCLE_RADIUS)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()


pygame 2.3.0 (SDL 2.24.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# Game updates

# use mediapipe to speer a 'gripper' from the right and left hand
# some circles are larger and harder to drag
# click first to reval the circle size, 
# the number of clicks is listed on the circle 

# the game instructs how to pinch the circle to drag it
# haptic feedback when the circle is successfully placed in the square
# some circles need two hands to drag



: 

In [ ]:
# Make Sounds: 

import numpy as np
from scipy.io.wavfile import write

sample_rate = 44100  # CD quality
duration = 0.2  # seconds
t = np.linspace(0, duration, int(sample_rate * duration), False)

# Make a frequency sweep down from 1200Hz to 800Hz
start_freq = 1200
end_freq = 800
frequencies = np.linspace(start_freq, end_freq, t.size)
waveform = np.sin(2 * np.pi * frequencies * t)

# Add quick decay to sound like a laser
envelope = np.exp(-20 * t)
waveform *= envelope

# Normalize to 16-bit PCM range
waveform_int16 = np.int16(waveform / np.max(np.abs(waveform)) * 32767)

write("laser.wav", sample_rate, waveform_int16)
print("Laser sound saved to laser.wav!")



In [ ]:
# Simple space game:

import cv2
import mediapipe as mp
import pygame
import math

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

pygame.init()
screen_width, screen_height = 640, 480
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Triangle with Correct Laser Direction")

pygame.mixer.init()
laser_sound = pygame.mixer.Sound("laser.wav")

cap = cv2.VideoCapture(0)
clock = pygame.time.Clock()
running = True

triangle_pos = [screen_width // 2, screen_height //2]
laser_speed = 10
lasers = []
triggered = False
size = 50
smoothed_angle = 0
angle_alpha = 0.2
min_dist = 0.02
max_dist = 0.15

while running:
    corrected_angle = smoothed_angle

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)
    angle = smoothed_angle

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            index_tip = hand_landmarks.landmark[8]
            index_mcp = hand_landmarks.landmark[5]

            cx = int(index_tip.x * screen_width)
            cy = int(index_tip.y * screen_height)
            triangle_pos[0], triangle_pos[1] = cx, cy

            dist = math.hypot(index_tip.x - index_mcp.x, index_tip.y - index_mcp.y)
            norm_dist = (dist - min_dist) / (max_dist - min_dist)
            norm_dist = max(0.0, min(1.0, norm_dist))
            target_angle = norm_dist * 2 * math.pi
            smoothed_angle = angle_alpha * target_angle + (1 - angle_alpha) * smoothed_angle
            corrected_angle = smoothed_angle

            # Rotated tip & laser direction from same math
            tip_offset_x = size * math.cos(corrected_angle)
            tip_offset_y = size * math.sin(corrected_angle)
            rotated_tip_x = cx + tip_offset_x
            rotated_tip_y = cy + tip_offset_y

            norm = math.hypot(tip_offset_x, tip_offset_y)
            dx_laser = (tip_offset_x / norm) * laser_speed
            dy_laser = (tip_offset_y / norm) * laser_speed

            thumb_tip = hand_landmarks.landmark[4]
            dist_thumb = math.hypot(thumb_tip.x - index_mcp.x, thumb_tip.y - index_mcp.y)
            threshold = 0.1

            if dist_thumb < threshold:
                if not triggered:
                    lasers.append({'pos': [rotated_tip_x, rotated_tip_y], 'dx': dx_laser, 'dy': dy_laser})
                    laser_sound.play()
                    triggered = True
            else:
                triggered = False

    for laser in lasers:
        laser['pos'][0] += laser['dx']
        laser['pos'][1] += laser['dy']

    lasers = [l for l in lasers if 0 <= l['pos'][0] <= screen_width and 0 <= l['pos'][1] <= screen_height]
    screen.fill((255, 255, 255))

    points = [
        (size, 0),
        (-size // 2, size // 2),
        (-size // 2, -size // 2),
    ]

    rotated_points = []
    for px, py in points:
        rx = px * math.cos(corrected_angle) - py * math.sin(corrected_angle)
        ry = px * math.sin(corrected_angle) + py * math.cos(corrected_angle)
        rotated_points.append((cx + rx, cy + ry))

    pygame.draw.polygon(screen, (0, 0, 0), rotated_points)

    for laser in lasers:
        pygame.draw.line(screen, (255, 0, 0), laser['pos'],
                         (laser['pos'][0] + laser['dx'] * 2, laser['pos'][1] + laser['dy'] * 2), 3)

    pygame.display.flip()
    clock.tick(30)
    cv2.imshow("Webcam", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.quit()


: 

laser.wav created successfully!
